In [1]:
# Import des modules
import pandas as pd
import numpy as np

import csv

In [2]:
# Import des données pour les questions avec Pandas, avec formatage de la date

df_players = pd.read_csv("donnees_football/Player.csv")
df_player_att = pd.read_csv("donnees_football/Player_Attributes.csv", parse_dates=["date"])
df_teams = pd.read_csv("donnees_football/Team.csv")
df_matchs = pd.read_csv("donnees_football/Match.csv", parse_dates=["date"])

In [3]:
"""Question 1 : Déterminez le(s) match(s) avec la plus grande différence de buts
dans le jeu de données."""

def question1():
    """Donne le(s) match(s) avec la plus grande différence de buts
    dans le jeu de données : équipes, score, date

    Returns
    -------
    list[str]
        Les matchs avec la plus grande différence de buts : équipes, score, date
    """

    df = df_matchs.copy()

    # Colonne différence de buts
    df["goal_diff"] = abs(df["home_team_goal"] - df["away_team_goal"])

    # Plus grande différence de buts
    max_goal_diff = max(df["goal_diff"])

    # Matchs cherchés
    res_match = df[df["goal_diff"] == max_goal_diff]

    # Fusion avec teams pour avoir le nom des équipes
    res_match = res_match.merge(
        df_teams[["team_api_id", "team_long_name"]],
        left_on="home_team_api_id",
        right_on="team_api_id",
        how="left"
    ).merge(
        df_teams[["team_api_id", "team_long_name"]],
        left_on="away_team_api_id",
        right_on="team_api_id",
        how="left",
        suffixes=('_home', '_away')
    )

    # Résultat mis en forme
    res_beau = []
    for idx, row in res_match.iterrows():
        home_name = row["team_long_name_home"]
        away_name = row["team_long_name_away"]
        home_goal = row["home_team_goal"]
        away_goal = row["away_team_goal"]
        date = row["date"].date()

        res_beau.append(f"{home_name} {home_goal} - {away_goal} {away_name} ({date})")

    with open("question1.txt", "w", encoding="utf8") as f:
        f.write(str(res_beau))

    return res_beau

question1()

['PSV 10 - 0 Feyenoord (2010-10-24)']

In [4]:
"""Question 1 : Déterminez le(s) match(s) avec la plus grande différence de buts
dans le jeu de données."""

def question1_pypur():
    """Donne le(s) match(s) avec la plus grande différence de buts
    dans le jeu de données : équipes, score, date

    Returns
    -------
    list[str]
        Les matchs avec la plus grande différence de buts : équipes, score, date
    """

    with open("donnees_football\\Team.csv", mode="r", newline="", encoding="utf-8") as f:
        lecteur = csv.DictReader(f)
        data_teams = [ligne for ligne in lecteur]
    with open("donnees_football\\Match.csv", mode="r", newline="", encoding="utf-8") as f:
        lecteur = csv.DictReader(f)
        data_matchs = [ligne for ligne in lecteur]

    # Dictionnaire pour accéder aux noms d'équipes
    id_to_name = {team["team_api_id"]: team["team_long_name"] for team in data_teams}

    # Variable différence de buts
    for m in data_matchs:
        m["goal_diff"] = abs(int(m["home_team_goal"]) - int(m["away_team_goal"]))

    # Plus grande différence de buts
    max_diff = max(m["goal_diff"] for m in data_matchs)

    # Matchs cherchés
    res = []
    for m in data_matchs:
        if m["goal_diff"] == max_diff:
            home_name = id_to_name.get(m["home_team_api_id"], "?")
            away_name = id_to_name.get(m["away_team_api_id"], "?")
            desc = (
                f"{home_name} {m['home_team_goal']} - {m['away_team_goal']}" +
                f"{away_name} ({m['date']})"
            )
            res.append(desc)

    with open("question1_pypur.txt", "w", encoding="utf8") as f:
        f.write(str(res))

    return res

question1_pypur()

['PSV 10 - 0Feyenoord (2010-10-24 00:00:00)']

In [5]:
"""Question 6 : Quels sont les 10 joueurs ayant obtenu la
meilleure note (overall_rating) en 2015 (meilleure note de l’année
s’il y en a plusieurs) ?"""
# Note : on généralise en créant les paramètres année et nombre de joueurs à afficher

def question6(annee, n_joueurs):
        """
        Donne la liste des joueurs ayant obtenu la meilleure note (overall_rating)
        pour une certaine année

        Parameters
        ----------
        annee : int
                L'année considérée
        n_joueurs : int
                Le nombre de joueurs à afficher dans le top

        Returns
        -------
        list[str]
                Les meilleurs joueurs : nom et rating

        """
        assert isinstance(n_joueurs, int)
        assert isinstance(annee, int)

        # annee en str pour pouvoir la joindre à un str
        annee = str(annee)

        df = df_player_att.copy()

        # Nettoyage
        df = df.dropna(subset=["player_api_id", "date", "overall_rating"])

        # On ne considère que l'année annee
        df_annee = df[
                (df["date"] >= (annee + "-01-01")) &
                (df["date"] < (annee + "-12-31"))
        ]

        # Comme il y a plusieurs enregistrements d'attributs pour un joueur par an,
        # pour chaque joueur, on ne garde que la meilleure note, et un seul
        # enregistrement
        df_meilleurs_annee = df_annee.groupby("player_api_id")["overall_rating"].max()
        df_annee = pd.merge(
                df_annee,
                df_meilleurs_annee,
                on=["player_api_id", "overall_rating"]
        )
        df_annee = df_annee.drop_duplicates(subset="player_api_id", keep="first")

        # Classement par note descendante
        df_annee = df_annee.sort_values("overall_rating", ascending=False)

        # Fusion avec informations sur les joueurs pour avoir leurs noms
        meilleurs_joueurs_annee = pd.merge(df_annee, df_players, on="player_api_id")

        # Top n_joueurs
        meilleurs_joueurs_annee = meilleurs_joueurs_annee[:n_joueurs]

        # Résultat mis en forme
        res_beau = []
        for idx, row in meilleurs_joueurs_annee.iterrows():
                player_name = row["player_name"]
                rating = row["overall_rating"]
                res_beau.append(
                        f"{player_name} ({rating})"
                )

        with open("question6.txt", "w", encoding="utf8") as f:
                f.write(str(res_beau))

        return res_beau

print(question6(2015, 10))


['Lionel Messi (94.0)', 'Cristiano Ronaldo (93.0)', 'Arjen Robben (90.0)', 'Luis Suarez (90.0)', 'Manuel Neuer (90.0)', 'Eden Hazard (89.0)', 'Neymar (89.0)', 'Zlatan Ibrahimovic (89.0)', 'Bastian Schweinsteiger (88.0)', 'David Silva (88.0)']


In [6]:
"""Question 7 : Déterminez le(s) match(s) avec la plus grande différence entre
les moyennes des notes des joueurs (overall_rating) des deux équipes."""

def question7():
    """Donne le(s) match(s) avec la plus grande différence entre
    les moyennes des notes des joueurs (overall_rating) des deux équipes.

    Returns
    -------
    list[str]
        Les matchs avec la plus grande différence entre les moyennes des notes
        des joueurs des deux équipes : équipes, notes, score, date, différence des notes
    """

    # On prépare l'historique des notes par joueur
    player_att = df_player_att.copy()
    player_att.sort_values(["player_api_id", "date"], inplace=True)

    # Dictionnaire : player_api_id -> liste (date, overall_rating)
    rating_history = (
        player_att.groupby("player_api_id")[["date", "overall_rating"]]
          .apply(lambda df: list(df.itertuples(index=False, name=None)))
          .to_dict()
    )

    def rating_before(pid, match_date):
        """Renvoie la note la plus récente avant match_date
        (ou None s'il n'y en a pas)"""
        hist = rating_history.get(pid, [])
        # on parcourt en sens inverse pour trouver la dernière note valide
        for date, rating in reversed(hist):
            if date < match_date:
                return rating
        return None

    # Liste des variables des 11 joueurs de chaque équipe
    home_cols = [f"home_player_{i}" for i in range(1, 12)]
    away_cols = [f"away_player_{i}" for i in range(1, 12)]

    # Pour chaque match, on calcule la moyenne des notes

    matchs = df_matchs.copy()

    # Nettoyage
    matchs = matchs.dropna(subset=home_cols + away_cols)

    def team_rating_moy(row, player_cols):
        """Moyenne de la note d'une équipe pour un match (on ne la retient que si
        tous les joueurs ont un rating)"""
        md = row["date"]
        ratings = []
        for pid in row[player_cols]:
            rating = rating_before(pid, md)
            if rating is None:
                return None
            ratings.append(rating)
        return np.mean(ratings)

    matchs["home_rating"] = matchs.apply(lambda r: team_rating_moy(r, home_cols), axis=1)
    matchs["away_rating"] = matchs.apply(lambda r: team_rating_moy(r, away_cols), axis=1)

    # Colonne différence des ratings
    matchs["rating_diff"] = abs(matchs["home_rating"] - matchs["away_rating"])
    max_diff = matchs["rating_diff"].max()
    res = matchs[matchs["rating_diff"] == max_diff]

    # Fusion avec teams pour avoir le nom des équipes
    res = res.merge(
            df_teams[["team_api_id", "team_long_name"]],
            left_on="home_team_api_id", right_on="team_api_id",
            how="left"
        ).merge(
            df_teams[["team_api_id", "team_long_name"]],
            left_on="away_team_api_id", right_on="team_api_id",
            how="left", suffixes=("_home", "_away")
        )

    # Résultat mis en forme
    res_beau = []
    for idx, row in res.iterrows():
        home = row["team_long_name_home"]
        away = row["team_long_name_away"]
        home_goal = row["home_team_goal"]
        away_goal = row["away_team_goal"]
        date = row["date"].date()
        home_rating = row["home_rating"]
        away_rating = row["away_rating"]
        diff = row["rating_diff"]
        res_beau.append(
            f"{home} (note : {home_rating:.2f}) {home_goal} - {away_goal} " +
            f"{away} (note : {away_rating:.2f}) ({date}) " +
            f"| Différence des notes : {diff:.2f}"
        )

    with open("question7.txt", "w", encoding="utf8") as f:
        f.write(str(res_beau))

    return res_beau

print(question7())


['FC Barcelona (note : 86.45) 3 - 0 Racing Santander (note : 67.55) (2011-10-15) | Différence des notes : 18.91']
